In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from pandas import DataFrame,Timestamp,concat,read_csv,to_datetime,NA
from plotly.graph_objects import Bar,Box,Candlestick,Figure,Scatter
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler


###
# Utils
colors = {
    'background_maindiv': '#F3FF33',
    'background_plot': '#B4BD26'
}
###

# Create a Dash web application
app = dash.Dash(__name__)
# Define the layout of the app
app.layout = html.Div(style={'backgroundColor': colors['background_maindiv']}, children=[
    html.H1(children='Stock Dashboard', style={'textAlign': 'center', 'font-family':'Helve',"fontSize": "48px"}),
    html.Div([
      dcc.Dropdown(['AAPL', 'AMZN', 'GOOG'], value='AAPL', id='selected-stock')
    ], style={'width': '100%', 'display': 'inline-block', 'verticalAlign': 'middle'}),
    html.Div([
        dcc.Graph(figure={}, id='our-first-graph', style={'display': 'inline-block'}),
        dcc.Graph(figure={}, id='our-second-graph', style={'display': 'inline-block'})
    ], style={'display': 'inline-block', 'verticalAlign': 'middle'}),
    html.Div([
      dcc.Graph(figure={}, id='our-third-graph')
    ], style={'width': '100%', 'display': 'inline-block', 'verticalAlign': 'middle'})
])
@app.callback(
    Output(component_id='our-first-graph', component_property='figure'),
    Output(component_id='our-second-graph', component_property='figure'),
    Output(component_id='our-third-graph', component_property='figure'),
    [
        Input(component_id="selected-stock", component_property='value')
    ]
)
def update_output(selected_stock_value):
  ###################################################################################
  # DATA PROCESSING
  datecols = ["Calc Date","Period End Date"]
  funda_orig_df = read_csv(filepath_or_buffer=f"{selected_stock_value}_funda.csv")
  funda_orig_df.drop(columns=['Unnamed: 0'], inplace=True)
  for datecol in datecols:
    funda_orig_df[datecol] = to_datetime(funda_orig_df[datecol])
    
  funda_df = funda_orig_df.copy()
  funda_df.drop(columns=["Instrument","Period End Date"], inplace=True)
  funda_df.set_index(keys=["Calc Date"], inplace=True)
  tmpadd = DataFrame(data={'Earnings before Interest & Taxes (EBIT)':[NA]}, index=[Timestamp.today()])
  tmpadd.index.rename(name='Calc Date', inplace=True)
  funda_df = concat(objs=[funda_df,tmpadd], axis=0)
  funda_df = funda_df.resample(rule='min').ffill().copy()
  funda_df.dropna(inplace=True)
  funda_df.index.rename(name="Date", inplace=True)

  funda2_orig_df = read_csv(filepath_or_buffer=f"{selected_stock_value}_funda2.csv")
  funda2_orig_df.drop(columns=["Instrument"], inplace=True)
  funda2_orig_df["Date"] = to_datetime(funda2_orig_df["Date"])
  funda2_orig_df.drop_duplicates(subset=["Date"], keep='first', inplace=True)
  funda2_df = funda2_orig_df.copy()
  funda2_df.set_index(keys=["Date"], inplace=True)
  funda2_df = funda2_df.resample(rule='min').ffill().copy()
  funda2_df.index = funda2_df.index.tz_convert(tz=None)

  price_df1 = read_csv(filepath_or_buffer=f"{selected_stock_value}_2023_07_01_2023_09_30.csv", index_col='Date', parse_dates=["Date"])
  price_df2 = read_csv(filepath_or_buffer=f"{selected_stock_value}_2023_10_01_2023_12_31.csv", index_col='Date', parse_dates=["Date"])
  price_df3 = read_csv(filepath_or_buffer=f"{selected_stock_value}_2024_01_01_2024_03_06.csv", index_col='Date', parse_dates=["Date"])
  price_df4 = read_csv(filepath_or_buffer=f"{selected_stock_value}_2024_03_07_2024_05_28.csv", index_col='Date', parse_dates=["Date"])
  price_df5 = read_csv(filepath_or_buffer=f"{selected_stock_value}_2024_05_29_2024_07_04.csv", index_col='Date', parse_dates=["Date"])
  price_df = concat(objs=[price_df1,price_df2,price_df3,price_df4,price_df5], axis=0)
  
  
  fdf = concat(objs=[price_df,funda_df,funda2_df], axis=1)
  fdf.dropna(inplace=True)
  fdf = fdf.tail(n=10000).copy()
  ###################################################################################

  ###################################################################################
  # First Chart
  top_left_plot = Figure(data=[Candlestick(x=fdf.index, open=fdf['OPEN'], high=fdf['HIGH'], low=fdf['LOW'], close=fdf['CLOSE'], name='price')])
  top_left_plot.update_layout(xaxis_rangeslider_visible=False)
  ###################################################################################

  ###################################################################################
  # Third Chart
  scaler = StandardScaler()
  pdf = DataFrame(scaler.fit_transform(fdf), columns=fdf.columns, index=fdf.index)
  bottom_plot = Figure()
  for col in ['CLOSE','Earnings before Interest & Taxes (EBIT)','P/E (Daily Time Series Ratio)']:
    bottom_plot.add_trace(trace=Scatter(x=pdf.index, y=pdf[col], name=col))
  ###################################################################################

  ###################################################################################
  # Additional Data Processing for Top Right Chart
  funda_orig_df['year'] = funda_orig_df["Period End Date"].dt.year.astype(str).str[2:]
  funda_orig_df['quarter'] = funda_orig_df["Period End Date"].dt.quarter.astype(str)
  funda_orig_df['quarter_str'] = 'Q'+funda_orig_df['quarter']+"'"+funda_orig_df['year']
  funda_orig_df.drop(columns=['year','quarter'], inplace=True)
  funda_orig_df.rename(columns={'quarter_str':'quarter'}, inplace=True)

  funda2_orig_df['year'] = funda2_orig_df["Date"].dt.year.astype(str).str[2:]
  funda2_orig_df['quarter'] = funda2_orig_df["Date"].dt.quarter.astype(str)
  funda2_orig_df['quarter_str'] = 'Q'+funda2_orig_df['quarter']+"'"+funda2_orig_df['year']
  funda2_orig_df.drop(columns=['year','quarter'], inplace=True)
  funda2_orig_df.rename(columns={'quarter_str':'quarter'}, inplace=True)
  ###################################################################################

  ###################################################################################
  # Second Chart
  top_right_plot = make_subplots(specs=[[{"secondary_y": True}]])
  top_right_plot.add_trace(Bar(name="EBIT", x=funda_orig_df["quarter"], y=funda_orig_df["Earnings before Interest & Taxes (EBIT)"]), secondary_y=False)
  for quarter in funda2_orig_df.quarter.unique():
    top_right_plot.add_trace(trace=Box(y=funda2_orig_df[funda2_orig_df['quarter']==quarter]['P/E (Daily Time Series Ratio)'], name=quarter, showlegend=False),secondary_y=True)
  ###################################################################################

  return top_left_plot,top_right_plot,bottom_plot


if __name__ == '__main__':
    app.run_server(mode='inline')  # Run the app in the Jupyter Notebook inline mode